# Семинар № 7 - Задача распознавания# Content

### 1.EDA
### 2. Writing a custom dataset for our work
### 3. Define the model
### 4. Finetuning from a pretrained model
### 5. Prediction
### 6. End Note

# 1.EDA

### Install and import necessary libraries

In [ ]:
import os
import re
import random

import cv2
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
set_seed(42)

##  Load train and test file

In [ ]:
train_df = pd.read_csv("global-wheat-detection/train.csv")
submit = pd.read_csv("global-wheat-detection/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
train_df=train_df.drop(columns=['width','height','source']) #Drop unwanted columns

In [ ]:
train_df['image_id'].nunique() # There are total 3373 unique image in training dataset

In [ ]:
(train_df['image_id'].value_counts()).max()  # maximum number of boxes in a single image are 116

In [ ]:
(train_df['image_id'].value_counts()). min() # Minimum number of box in a single image is 1

### Splitting the dimension of box in the formate [xmin, ymin, w, h]
#### Latter on we will convert the deminsion of box into [xmin, ymin, xmax, ymax] in 

In [ ]:
train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

In [ ]:
train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x))) ##Lets convert the Box in 
train_df['x'] = train_df['x'].astype(np.float32)                                        #in our desired formate    
train_df['y'] = train_df['y'].astype(np.float32)
train_df['w'] = train_df['w'].astype(np.float32)
train_df['h'] = train_df['h'].astype(np.float32)

In [ ]:
train_df.head() 

In [ ]:
submit[['x', 'y', 'w', 'h']] = np.stack(submit['PredictionString'].apply(lambda x: [0, 0, 1, 1]))

In [ ]:
submit.head()

### Splitting the data into train and validation set

In [ ]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

# 2.Writing a custom dataset for our work

### 2.1 Writing a custom dataset for train and validation images

In [ ]:
class WheatDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None,train=True):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.train=train

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        records = self.df[self.df['image_id'] == image_id]   
        boxes = records[['x', 'y', 'w', 'h']].values
        # throw to xyxy
        pass

        # there is only one class
        labels = None
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        if self.transforms:
            pass
        
        return image, target, image_id

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            None,
            ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )


def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
train_dir = 'global-wheat-detection/train'
test_dir = 'global-wheat-detection/test'

In [ ]:
class Averager:      ##Return the average loss 
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
        
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(train_df, train_dir, get_train_transforms(), True)
valid_dataset = WheatDataset(valid_df, train_dir, get_valid_transforms(), True)


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

### Lets visualize some of the images with bounding box

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

idx = 2
boxes = targets[idx]['boxes'].cpu().numpy().astype(int)
sample = images[idx].permute(1,2,0).cpu().numpy()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    sample = cv2.rectangle(sample.copy(), (box[0], box[1]), (box[2], box[3]), (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

# 3.Finetuuning the model

### Defining the model

Faster R-CNN is a model that predicts both bounding boxes and class scores for potential objects in the image.
![image.png](attachment:image.png)


Let’s explain how this architecture works, Faster RCNN is composed from 3 parts

1. Part 1 : Convolution layers : A CNN architecture is formed by a stack of distinct layers that transform the input volume into an output volume (e.g. holding the class scores) through a differentiable function.Convolutional networks were inspired by biological processes in that the connectivity pattern between neurons resembles the organization of the animal visual cortex. Individual cortical neurons respond to stimuli only in a restricted region of the visual field known as the receptive field. The receptive fields of different neurons partially overlap such that they cover the entire visual field.

2. Part 2 : Region Proposel Network (RPN) : RPN is small neural network sliding on the last feature map of the convolution layers and predict wether there is an object or not and also predict the bounding box of those objects.

3. Part 3 : Classes and Bounding Boxes prediction : Now we use another Fully connected neural networks that takes as an inpt the regions proposed by the RPN and predict object class ( classification) and Bounding boxes (Regression).

In [ ]:
# load a model; pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, pretrained_backbone=True)

In [ ]:
num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

### Lets train our model

In [ ]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

num_epochs = 5

loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in tqdm(train_data_loader):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)   ##Return the loss

        losses = sum(loss for loss in loss_dict.values()).type(torch.float32)
        loss_value = losses.item()

        loss_hist.send(loss_value)  #Average out the loss

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")

# 4. Prediction

### Lets load the test data

In [ ]:
test_dataset = WheatDataset(submit, test_dir, get_valid_transforms(), False)

In [ ]:
test_data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)  ##Test dataloader

### Set the threshold value for predicting bounding box.

In [ ]:
detection_threshold = 0.45

In [ ]:
## Lets make the prediction
results = []
model.eval()

for images, _, image_ids in test_data_loader:    

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()    ##Formate of the output's box is [Xmin,Ymin,Xmax,Ymax]
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32) #Compare the score of output with the threshold and
        scores = scores[scores >= detection_threshold]                    #slelect only those boxes whose score is greater
                                                                          # than threshold value
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]         
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]         #Convert the box formate to [Xmin,Ymin,W,H]

In [ ]:
sample = images[1].permute(1, 2, 0).cpu().numpy()
boxes = outputs[1]['boxes'].data.cpu().numpy()
scores = outputs[1]['scores'].data.cpu().numpy()

boxes = boxes[scores >= detection_threshold].astype(np.int32)

### Lets plot some of our prediction

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    sample = cv2.rectangle(sample.copy(),
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 2)
    
ax.set_axis_off()
ax.imshow(sample)